<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250716/%E3%83%93%E3%83%B3%E3%83%91%E3%83%83%E3%82%AD%E3%83%B3%E3%82%B0%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備


In [1]:
%%capture
# python-mip ライブラリをインストールする
!pip install mip

-----

## 演習問題1

以下の条件でビンパッキング問題を解く。

  * アイテムのサイズ: $W = [2, 5, 4, 7, 1, 3, 8]$
  * ビンの容量: $C = 10$

###数理モデルでの定式化

この問題は、標準的なビンパッキング問題として定式化できます。

  * **パラメータ**:
      * アイテムの集合
   $I = \{0, 1, ..., 6\}$
      * アイテムのサイズ $w\_i \\in W$
      * ビンの容量 $C = 10$
      * アイテム数 $n = 7$
      * 利用可能なビンの最大数 $m = n = 7$

  * **決定変数**:

      *
  $x_{ij} \in \{0, 1\}$: アイテム $i$ をビン $j$ に入れる場合に1、そうでない場合に0となるバイナリ変数。
      *
  $y_j \in \{0, 1\}$: ビン $j$ を使用する場合に1、そうでない場合に0となるバイナリ変数。

  * **目的関数**:
    使用するビンの総数を最小化します。
    $$\min \sum_{j=0}^{m-1} y_j$$

  * **制約条件**:

    1.  **アイテム割り当て制約**: 各アイテムは、必ずいずれか1つのビンに割り当てられます。
        $$\sum_{j=0}^{m-1} x_{ij} = 1 \quad (\forall i \in I)$$
    2.  **容量制約**: 各ビンに入れられたアイテムのサイズの合計は、そのビンの容量を超えてはならず、アイテムは使用されるビンにのみ入れられます。
        $$\sum_{i=0}^{n-1} w_i x_{ij} \le C \cdot y_j \quad (\forall j \in \{0, 1, ..., m-1\})$$

###Python (MIP) による実装


In [2]:
import mip

# --- 演習問題1 データ ---
# アイテムのサイズ
item_weights = [2, 5, 4, 7, 1, 3, 8]
# ビンの容量
bin_capacity = 10
# アイテムの数
num_items = len(item_weights)
# 利用可能なビンの最大数（アイテム数と同じにする）
num_bins = num_items

# --- モデルの作成 ---
# モデルオブジェクトを最小化問題として作成
model = mip.Model(name="bin_packing_1", sense=mip.MINIMIZE)

# --- 変数の定義 ---
# x[i][j]: アイテムiをビンjに入れる場合に1
x = [[model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for j in range(num_bins)] for i in range(num_items)]
# y[j]: ビンjを使用する場合に1
y = [model.add_var(var_type=mip.BINARY, name=f"y_{j}") for j in range(num_bins)]

# --- 目的関数の設定 ---
# 使用するビンの総数（y_jが1になるものの合計）を最小化
model.objective = mip.xsum(y[j] for j in range(num_bins))

# --- 制約条件の追加 ---
# 制約1: 各アイテムは正確に一つのビンに割り当てられる
for i in range(num_items):
    model += mip.xsum(x[i][j] for j in range(num_bins)) == 1, f"item_assignment_{i}"

# 制約2: 各ビンの容量制約
for j in range(num_bins):
    model += mip.xsum(item_weights[i] * x[i][j] for i in range(num_items)) <= bin_capacity * y[j], f"capacity_{j}"

# --- 問題の求解 ---
print("\nモデルの最適化を開始します...")
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小ビン数: {int(model.objective_value)}")
    print("\n各ビンの内容:")
    for j in range(num_bins):
        # y_j.x は変数の値（0か1）を取得する。浮動小数点数の誤差を考慮し、0.99以上を1とみなす
        if y[j].x >= 0.99:
            bin_items = []
            bin_weight = 0
            for i in range(num_items):
                if x[i][j].x >= 0.99:
                    bin_items.append(item_weights[i])
                    bin_weight += item_weights[i]
            print(f"  ビン {j}: アイテム {bin_items}, 合計サイズ: {bin_weight}")
elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解が存在しません。")
else:
    print(f"最適化ステータス: {status}")


モデルの最適化を開始します...
最小ビン数: 3

各ビンの内容:
  ビン 1: アイテム [5, 4, 1], 合計サイズ: 10
  ビン 2: アイテム [2, 8], 合計サイズ: 10
  ビン 3: アイテム [7, 3], 合計サイズ: 10


-----

## 演習問題2

演習問題1と同じアイテムで、ビンの容量を変更した問題を解きます。

  * アイテムのサイズ: $W = [2, 5, 4, 7, 1, 3, 8]$
  * ビンの容量: $C = 12$

###数理モデルでの定式化

モデルの構造は演習問題1と同一で、パラメータ $C$ のみが変更されます。

###Python (MIP) による実装


In [4]:
import mip

# --- 演習問題3 データ ---
# アイテムのサイズ
item_weights = [2, 5, 4, 7, 1, 3, 8]
# ビンの容量
bin_capacity = 12
# アイテムの数
num_items = len(item_weights)
# 利用可能なビンの最大数
num_bins = num_items

# --- モデルの作成 ---
model = mip.Model(name="bin_packing_3", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x = [[model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for j in range(num_bins)] for i in range(num_items)]
y = [model.add_var(var_type=mip.BINARY, name=f"y_{j}") for j in range(num_bins)]

# --- 目的関数の設定 ---
model.objective = mip.xsum(y[j] for j in range(num_bins))

# --- 制約条件の追加 ---
# 制約1: 各アイテムは正確に一つのビンに割り当てられる
for i in range(num_items):
    model += mip.xsum(x[i][j] for j in range(num_bins)) == 1, f"item_assignment_{i}"

# 制約2: 各ビンの容量制約
for j in range(num_bins):
    model += mip.xsum(item_weights[i] * x[i][j] for i in range(num_items)) <= bin_capacity * y[j], f"capacity_{j}"

# --- 問題の求解 ---
print("\nモデルの最適化を開始します...")
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小ビン数: {int(model.objective_value)}")
    print("\n各ビンの内容:")
    for j in range(num_bins):
        if y[j].x >= 0.99:
            bin_items = []
            bin_weight = 0
            for i in range(num_items):
                if x[i][j].x >= 0.99:
                    bin_items.append(item_weights[i])
                    bin_weight += item_weights[i]
            print(f"  ビン {j}: アイテム {bin_items}, 合計サイズ: {bin_weight}")
elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解が存在しません。")
else:
    print(f"最適化ステータス: {status}")


モデルの最適化を開始します...
最小ビン数: 3

各ビンの内容:
  ビン 0: アイテム [1, 3, 8], 合計サイズ: 12
  ビン 3: アイテム [4, 7], 合計サイズ: 11
  ビン 6: アイテム [2, 5], 合計サイズ: 7


-----

## 演習問題3

演習問題1の条件に、以下の追加制約を加えた問題を解く

  * アイテムのサイズ: $W = [2, 5, 4, 7, 1, 3, 8]$
  * ビンの容量: $C = 10$
  * **追加制約**: アイテム0 (サイズ2) とアイテム6 (サイズ8) は、同じビンに入れてはならない。

###数理モデルでの定式化

基本モデルに、以下の新しい制約を追加する。

  * **追加制約（非同居制約）**:
    任意のビン $j$ において、アイテム0とアイテム6が同時に存在することはできません。これは、変数 $x\_{0j}$ と $x\_{6j}$ が同時に1になることを防ぐことで表現できます。
    $$x_{0j} + x_{6j} \le 1 \quad (\forall j \in \{0, 1, ..., m-1\})$$
    この制約により、各ビン $j$ にはアイテム0かアイテム6のどちらか一方、もしくはいずれも入らない、という状態が保証されます。

###Python (MIP) による実装


In [5]:
import mip

# --- 演習問題4 データ ---
# アイテムのサイズ
item_weights = [2, 5, 4, 7, 1, 3, 8]
# ビンの容量
bin_capacity = 10
# アイテムの数
num_items = len(item_weights)
# 利用可能なビンの最大数
num_bins = num_items

# 非同居制約の対象アイテムのインデックス
item_a_idx = 0
item_b_idx = 6

# --- モデルの作成 ---
model = mip.Model(name="bin_packing_4", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x = [[model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for j in range(num_bins)] for i in range(num_items)]
y = [model.add_var(var_type=mip.BINARY, name=f"y_{j}") for j in range(num_bins)]

# --- 目的関数の設定 ---
model.objective = mip.xsum(y[j] for j in range(num_bins))

# --- 制約条件の追加 ---
# 制約1: 各アイテムは正確に一つのビンに割り当てられる
for i in range(num_items):
    model += mip.xsum(x[i][j] for j in range(num_bins)) == 1, f"item_assignment_{i}"

# 制約2: 各ビンの容量制約
for j in range(num_bins):
    model += mip.xsum(item_weights[i] * x[i][j] for i in range(num_items)) <= bin_capacity * y[j], f"capacity_{j}"

# ★追加制約3: 特定アイテムの非同居制約
for j in range(num_bins):
    model += x[item_a_idx][j] + x[item_b_idx][j] <= 1, f"incompatible_{item_a_idx}_{item_b_idx}_in_bin_{j}"

# --- 問題の求解 ---
print("\nモデルの最適化を開始します...")
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小ビン数: {int(model.objective_value)}")
    print("\n各ビンの内容:")
    for j in range(num_bins):
        if y[j].x >= 0.99:
            bin_items = []
            bin_weight = 0
            for i in range(num_items):
                if x[i][j].x >= 0.99:
                    bin_items.append(item_weights[i])
                    bin_weight += item_weights[i]
            print(f"  ビン {j}: アイテム {bin_items}, 合計サイズ: {bin_weight}")
else:
    print(f"最適化ステータス: {status}")


モデルの最適化を開始します...
最小ビン数: 4

各ビンの内容:
  ビン 0: アイテム [1, 8], 合計サイズ: 9
  ビン 1: アイテム [4], 合計サイズ: 4
  ビン 5: アイテム [2, 5, 3], 合計サイズ: 10
  ビン 6: アイテム [7], 合計サイズ: 7


-----
## 演習問題4

演習問題1の条件に、以下の追加制約を加えた問題を解きます。

  * アイテムのサイズ: $W = [2, 5, 4, 7, 1, 3, 8]$
  * ビンの容量: $C = 10$
  * **追加制約**: 各ビンに入れられるアイテムの最大数は2個

###数理モデルでの定式化

基本モデルに、ビンごとのアイテム数を制限する制約を追加します。

  * **追加制約（最大アイテム数制約）**:
  任意のビン $j$ に入っているアイテムの総数は、最大で2個までです。ビン $j$ に入っているアイテムの数は $\sum_{i=0}^{n-1} x_{ij}$ で計算できます。
  $$\sum_{i=0}^{n-1} x_{ij} \le 2 \cdot y_j \quad (\forall j \in \{0, 1, ..., m-1\})$$
  $y_j$ を右辺に含めることで、この制約が使用されるビン ($y_j=1$) に対してのみ有効になるようにしています。もしビンが使われなければ ($y_j=0$)、左辺は0となり、対応する $x_{ij}$ も0となる。

###Python (MIP) による実装


In [6]:
import mip

# --- 演習問題5 データ ---
# アイテムのサイズ
item_weights = [2, 5, 4, 7, 1, 3, 8]
# ビンの容量
bin_capacity = 10
# アイテムの数
num_items = len(item_weights)
# 利用可能なビンの最大数
num_bins = num_items
# 各ビンの最大アイテム数
max_items_in_bin = 2

# --- モデルの作成 ---
model = mip.Model(name="bin_packing_5", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x = [[model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for j in range(num_bins)] for i in range(num_items)]
y = [model.add_var(var_type=mip.BINARY, name=f"y_{j}") for j in range(num_bins)]

# --- 目的関数の設定 ---
model.objective = mip.xsum(y[j] for j in range(num_bins))

# --- 制約条件の追加 ---
# 制約1: 各アイテムは正確に一つのビンに割り当てられる
for i in range(num_items):
    model += mip.xsum(x[i][j] for j in range(num_bins)) == 1, f"item_assignment_{i}"

# 制約2: 各ビンの容量制約
for j in range(num_bins):
    model += mip.xsum(item_weights[i] * x[i][j] for i in range(num_items)) <= bin_capacity * y[j], f"capacity_{j}"

# ★追加制約3: 各ビンの最大アイテム数制約
for j in range(num_bins):
    model += mip.xsum(x[i][j] for i in range(num_items)) <= max_items_in_bin * y[j], f"cardinality_{j}"

# --- 問題の求解 ---
print("\nモデルの最適化を開始します...")
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小ビン数: {int(model.objective_value)}")
    print("\n各ビンの内容:")
    for j in range(num_bins):
        if y[j].x >= 0.99:
            bin_items = []
            bin_weight = 0
            for i in range(num_items):
                if x[i][j].x >= 0.99:
                    bin_items.append(item_weights[i])
                    bin_weight += item_weights[i]
            print(f"  ビン {j}: アイテム {bin_items}, 合計サイズ: {bin_weight}")
else:
    print(f"最適化ステータス: {status}")


モデルの最適化を開始します...
最小ビン数: 4

各ビンの内容:
  ビン 1: アイテム [8], 合計サイズ: 8
  ビン 2: アイテム [2, 7], 合計サイズ: 9
  ビン 4: アイテム [5, 3], 合計サイズ: 8
  ビン 6: アイテム [4, 1], 合計サイズ: 5


-----
## 演習問題5

アイテムにカテゴリが設定され、特定のカテゴリのアイテムを同じビンに多数入れられない制約が加わります。

  * アイテムのサイズ: $W = [2, 5, 4, 7, 1, 3, 8]$
  * アイテムのカテゴリ: `["通常品", "壊れ物", "通常品", "壊れ物", "通常品", "通常品", "壊れ物"]`
  * ビンの容量: $C = 10$
  * **追加制約**: 各ビンに入れられる「壊れ物」カテゴリのアイテムは最大1個。

###数理モデルでの定式化

基本モデルに、カテゴリに関する制約を追加します。

  * **パラメータの追加**:
  $I_{fragile} \subseteq I$: 「壊れ物」カテゴリに属するアイテムのインデックスの集合。この問題では、$I_{fragile} = \{1, 3, 6\}$ となります。

  * **追加制約（カテゴリ制約）**:
  任意のビン $j$ に入っている「壊れ物」の総数は、最大で1個までです。
  $$\sum_{i \in I_{fragile}} x_{ij} \le 1 \times y_j \quad (\forall j \in \{0, 1, ..., m-1\})$$
  この制約により、ビン $j$ が使われていれば「壊れ物」が1つだけ入るか、全く入らないかのどちらかであることが保証されます。

### Python (MIP) による実装


In [8]:
import mip

# --- 演習問題6 データ ---
# アイテムのサイズ
item_weights = [2, 5, 4, 7, 1, 3, 8]
# アイテムのカテゴリ
categories = ["通常品", "壊れ物", "通常品", "壊れ物", "通常品", "通常品", "壊れ物"]
# ビンの容量
bin_capacity = 10
# アイテムの数
num_items = len(item_weights)
# 利用可能なビンの最大数
num_bins = num_items

# 「壊れ物」カテゴリのアイテムのインデックスを特定
fragile_item_indices = [i for i, category in enumerate(categories) if category == "壊れ物"]

# --- モデルの作成 ---
model = mip.Model(name="bin_packing_6", sense=mip.MINIMIZE)

# --- 変数の定義 ---
x = [[model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for j in range(num_bins)] for i in range(num_items)]
y = [model.add_var(var_type=mip.BINARY, name=f"y_{j}") for j in range(num_bins)]

# --- 目的関数の設定 ---
model.objective = mip.xsum(y[j] for j in range(num_bins))

# --- 制約条件の追加 ---
# 制約1: 各アイテムは正確に一つのビンに割り当てられる
for i in range(num_items):
    model += mip.xsum(x[i][j] for j in range(num_bins)) == 1, f"item_assignment_{i}"

# 制約2: 各ビンの容量制約
for j in range(num_bins):
    model += mip.xsum(item_weights[i] * x[i][j] for i in range(num_items)) <= bin_capacity * y[j], f"capacity_{j}"

# ★追加制約3: 各ビンに入れられる壊れ物の最大数制約
for j in range(num_bins):
    model += mip.xsum(x[i][j] for i in fragile_item_indices) <= y[j], f"fragile_limit_{j}"

# --- 問題の求解 ---
print("\nモデルの最適化を開始します...")
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小ビン数: {int(model.objective_value)}")
    print("\n各ビンの内容:")
    for j in range(num_bins):
        if y[j].x >= 0.99:
            bin_items = []
            bin_weight = 0
            item_indices = []
            for i in range(num_items):
                if x[i][j].x >= 0.99:
                    bin_items.append(f"{item_weights[i]} ({categories[i]})")
                    bin_weight += item_weights[i]
            print(f"  ビン {j}: アイテム {bin_items}, 合計サイズ: {bin_weight}")
else:
    print(f"最適化ステータス: {status}")


モデルの最適化を開始します...
最小ビン数: 3

各ビンの内容:
  ビン 1: アイテム ['2 (通常品)', '8 (壊れ物)'], 合計サイズ: 10
  ビン 2: アイテム ['7 (壊れ物)', '3 (通常品)'], 合計サイズ: 10
  ビン 4: アイテム ['5 (壊れ物)', '4 (通常品)', '1 (通常品)'], 合計サイズ: 10
